In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from PIL import Image
import io
import numpy as np
from matplotlib import pyplot 
import glob
import cv2
from skimage.metrics import structural_similarity as ssim

In [2]:
generatorPath = 'generator_6 (2).h5'
imagesValPath = 'images/val128'

mseList = []
ssimList = []

In [3]:
model = load_model(generatorPath)

In [4]:
# load an image
def load_image(pixels):
    # load image with the preferred size
    # convert to numpy array
    pixels = img_to_array(pixels)
    # scale from [0,255] to [-1,1]
    pixels = (pixels - 127.5) / 127.5
    # reshape to 1 sample
    pixels = np.expand_dims(pixels, 0)
    return pixels

In [5]:
def mse(imageA, imageB):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    # NOTE: the two images must have the same dimension
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])

    # return the MSE, the lower the error, the more "similar"
    # the two images are
    return err


def compare_images(imageA, imageB, title):
    # compute the mean squared error and structural similarity
    # index for the images
    m = mse(imageA, imageB)
    s = ssim(imageA, imageB)
    
    mseList.append(m)
    ssimList.append(s)
    
    '''
    # setup the figure
    fig = pyplot.figure(title)
    pyplot.suptitle("MSE: %.2f, SSIM: %.2f" % (m, s))
    # show first image
    ax = fig.add_subplot(1, 2, 1)
    pyplot.imshow(imageA, cmap = pyplot.cm.gray)
    pyplot.axis("off")
    # show the second image
    ax = fig.add_subplot(1, 2, 2)
    pyplot.imshow(imageB, cmap = pyplot.cm.gray)
    pyplot.axis("off")
    # show the images
    pyplot.show()
    '''

In [6]:
def average(lst):
    return sum(lst) / len(lst)

In [7]:
image_list = []
for filename in glob.glob(imagesValPath + '/*.png'):
    img = cv2.imread(filename)
    image_list.append(img)

sketchList = []
heightmapList = []
for image in image_list:
    width = img.shape[1]

    width_cutoff = width // 2
    s1 = image[:, :width_cutoff]
    s2 = image[:, width_cutoff:]
    sketchList.append(s1)
    heightmapList.append(s2)    

In [8]:
for i in range(len(sketchList)):
    sketch = sketchList[i]
    destination = heightmapList[i]
    
    sketch = load_image(sketch)
    
    gen_image = model.predict(sketch)
    gen_image = (gen_image + 1) / 2.0    

    gen_image = (gen_image[0] * 255.0).astype(np.uint8)
    
    example_target = cv2.cvtColor(destination, cv2.COLOR_BGR2GRAY)
    gen_image = cv2.cvtColor(gen_image, cv2.COLOR_BGR2GRAY)
    
    compare_images(gen_image, example_target, "test")

In [9]:
averageMse = average(mseList)
averageSsim = average(ssimList)

print(averageMse)
print(averageSsim)

15927.811354522704
0.02410199799539747
